In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = 'EleutherAI/pythia-160m'
model_name = 'EleutherAI/pythia-70m'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model_og = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
def get_input_after_tokenizer(inp: str):
		tokenizer = AutoTokenizer.from_pretrained(model_name)
		inp_out = tokenizer(inp, return_tensors="pt")
		print(inp_out)
		inputs = inp_out['input_ids'].squeeze(0)
		# print(inputs, inputs.shape)
		# TODO: IDK ABOUT WHATS GOING ON W/ TOKEN SIZE VS Vocab Size
		# Vocab size is 50204 and inp size is 50304
		one_hot = torch.zeros((inputs.shape[0], 50304), dtype=torch.int)
		for i in range(inputs.shape[0]):
			one_hot[i, inputs[i]] = 1
		# one_hot[inputs['input_ids'][0, 0]] = 1
		return one_hot#, inp_out['attention_mask']
get_input_after_tokenizer('hello world')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)

In [5]:
inps_one_hot = get_input_after_tokenizer("hello world")
seq_len = inps_one_hot.shape[0]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


In [6]:
model_og.gpt_neox.layers[0]

GPTNeoXLayer(
  (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_dropout): Dropout(p=0.0, inplace=False)
  (post_mlp_dropout): Dropout(p=0.0, inplace=False)
  (attention): GPTNeoXAttention(
    (rotary_emb): GPTNeoXRotaryEmbedding()
    (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
    (dense): Linear(in_features=512, out_features=512, bias=True)
    (attention_dropout): Dropout(p=0.0, inplace=False)
  )
  (mlp): GPTNeoXMLP(
    (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
    (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
    (act): GELUActivation()
  )
)

In [7]:
from simplified import ModelSel

model_sel = ModelSel(model_og)
model_sel

ModelSel(
  (embed_linear): Linear(in_features=50304, out_features=512, bias=False)
  (layer_norm): SimplfiedLayerNorm(
    (eps): Linear(in_features=512, out_features=512, bias=True)
    (ones_linear): Linear(in_features=512, out_features=512, bias=False)
    (ones_linear_neg): Linear(in_features=512, out_features=512, bias=False)
  )
  (attn): FixedAttentionMask(
    (attn): GPTNeoXAttention(
      (rotary_emb): GPTNeoXRotaryEmbedding()
      (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
      (dense): Linear(in_features=512, out_features=512, bias=True)
      (attention_dropout): Dropout(p=0.0, inplace=False)
    )
    (slice_query): Slicer(
      (slicer): Linear(in_features=192, out_features=64, bias=True)
    )
    (slice_value): Slicer(
      (slicer): Linear(in_features=192, out_features=64, bias=True)
    )
    (slice_key): Slicer(
      (slicer): Linear(in_features=192, out_features=64, bias=True)
    )
    (slice_rotary): Slicer(
      (slicer): Li

In [8]:
inps_one_hot_formatted = inps_one_hot.float()
# embed_linear.forward(inps_one_hot).shape
model_sel(inps_one_hot_formatted).shape

torch.Size([2, 1536])
QKV SIZE torch.Size([2, 1536]) torch.Size([2])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size(

torch.Size([2, 512])

In [2]:
inps_one_hot_formatted = inps_one_hot.float()
torch.onnx.export(model_sel, inps_one_hot_formatted,
                  'model_sel.onnx', verbose=True, opset_version=12)

NameError: name 'inps_one_hot' is not defined